In [7]:
import gzip
import json
import numpy as np
from collections import defaultdict

workpath="/data/jmoreno/Datasets/signalmedia/"

f=gzip.open(workpath+'signalmedia-1k.jsonl.gz','rb')


docs=[json.loads(s.decode('utf-8')) for s in f.readlines()]
titres=[x['title'] for x in docs]
content=[x['content'] for x in docs]
print(len(titres))

def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if x.isalpha() else " " for x in sent])
    sent = " ".join(sent.split())
    return sent


cleantitres=[sentense2cleanTokens(x) for x in titres]

1000


In [8]:
import nltk
onebigline=" ".join([line for line in cleantitres])
bgm = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(onebigline.split())
ignored_words = nltk.corpus.stopwords.words('english')
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
finder.nbest(bgm.raw_freq, 10)
finder.nbest(bgm.pmi, 10)
finder.nbest(bgm.likelihood_ratio, 10)
finder.nbest(bgm.chi_sq, 10)
finder.nbest(bgm.dice, 10)
finder.nbest(bgm.fisher, 10)
finder.nbest(bgm.jaccard, 10)
finder.nbest(bgm.mi_like, 10)
finder.nbest(bgm.poisson_stirling, 10)
finder.nbest(bgm.student_t, 10)
scored = finder.score_ngrams(bgm.pmi)

In [10]:
print(finder.nbest(bgm.pmi, 10))

[('aal', 'dryships'), ('abby', 'bishop'), ('abnormal', 'indications'), ('abu', 'dhabi'), ('access', 'pipeline'), ('accessing', 'europeans'), ('accident', 'importing'), ('acquisition', 'completes'), ('active', 'directory'), ('adidas', 'tubular')]


In [21]:
import spacy

nlp = spacy.load('en')

tokens = nlp(u'dog cat banana')

for token1 in tokens:
    print([token1.similarity(token2) for token2 in tokens])

[1.0, 0.53906965, 0.28761008]
[0.53906965, 1.0, 0.48752153]
[0.28761008, 0.48752153, 1.0]


In [22]:
dog=tokens[0]
print(dog.vector)

[  8.70676279e-01   2.25968742e+00  -6.31433249e-01   5.31767130e-01
   2.93478549e-01   1.71422410e+00  -3.22116327e+00   8.72992218e-01
   1.93997502e+00   2.87031293e+00   2.26860690e+00  -1.70549440e+00
   1.75721931e+00   1.00328475e-01  -1.84722948e+00   8.10127616e-01
  -2.36641622e+00  -6.74954116e-01  -2.57085586e+00   1.77607036e+00
  -1.85285389e-01   1.23867369e+00  -9.75624084e-01   1.93797398e+00
   2.00349092e-02  -1.60199952e+00  -5.47065377e-01   1.61460447e+00
  -2.17766643e-01  -2.21958590e+00  -7.38539815e-01  -6.19615197e-01
  -3.90584767e-01   1.81061625e-02  -5.94443269e-02  -1.39486265e+00
   1.60074508e+00  -5.22987843e-01   2.30428696e+00  -1.77107143e+00
  -1.88900435e+00  -2.22219491e+00   1.16671133e+00  -2.62894034e+00
  -2.51317739e+00  -6.32965326e-01  -2.00493455e+00  -7.31566668e-01
  -1.48643851e-01  -1.04217446e+00   3.28766465e+00  -2.02324653e+00
  -2.19653678e+00   3.25622559e+00   3.02244991e-01  -2.88570738e+00
   3.49860716e+00   3.15575647e+00

In [28]:
#!/usr/bin/env python
# coding: utf8
"""Train a multi-label convolutional neural network text classifier on the
IMDB dataset, using the TextCategorizer component. The dataset will be loaded
automatically via Thinc's built-in dataset loader. The model is added to
spacy.pipeline, and predictions are available via `doc.cats`. For more details,
see the documentation:
* Training: https://spacy.io/usage/training

Compatible with: spaCy v2.0.0+
"""
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets

import spacy
from spacy.util import minibatch, compounding


def main(model=None, output_dir=None, n_iter=3, n_texts=2000):
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")

    # add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    textcat = nlp.create_pipe('textcat')
    nlp.add_pipe(textcat, last=True)
    
    # add label to text classifier
    textcat.add_label('POSITIVE')

    # load the IMDB dataset
    print("Loading IMDB data...")
    (train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=n_texts)
    print("Using {} examples ({} training, {} evaluation)"
          .format(n_texts, len(train_texts), len(dev_texts)))
    train_data = list(zip(train_texts,
                          [{'cats': cats} for cats in train_cats]))
    print(train_texts[:5],train_cats[:5])
    optimizer = nlp.begin_training()
    print("Training the model...")
    print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))
    for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                           losses=losses)
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'  # print a simple table
                  .format(losses['textcat'], scores['textcat_p'],
                          scores['textcat_r'], scores['textcat_f']))

    # test the trained model
    test_text = "This movie sucked"
    doc = nlp(test_text)
    print(test_text, doc.cats)


def load_data(limit=0, split=0.8):
    """Load data from the IMDB dataset."""
    # Partition off part of the train data for evaluation
    train_data, _ = thinc.extra.datasets.imdb()
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{'POSITIVE': bool(y)} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])


def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}


if __name__ == '__main__':
    main()

Created blank 'en' model
Loading IMDB data...
Using 2000 examples (1600 training, 400 evaluation)
("This is absolutely the dumbest movie I've ever seen. What a waste of a splendid cast. That's James Cromwell as the ignoramus playing deputy. I could go on and on, but I would obviously be spending more time on this review than anybody ever did on the script. The only thing this movie is about is us vs. them and how to revel in profane slapstick beyond any reasonable human being's tolerance. This is one of the 10 worst movies I have ever seen -- and I LOVE James Garner.", "but I want to say I cannot agree more with Moira.\n\n\n\nWhat a wonderful film.\n\n\n\nI was thinking about it just this morning, wanting to give advice to some dopey sod who'd lost money on his debit card through fraud, and wanted to say 'Keep thy money in thine pocket' and realised I was talking like James Mason.\n\n\n\nEven tho he didn't say those words, I still think he would! I've never forgotten 'Are ye carrying?'

In [34]:
n_texts=2000

# load the IMDB dataset
print("Loading IMDB data...")
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=n_texts)
print("Using {} examples ({} training, {} evaluation)"
       .format(n_texts, len(train_texts), len(dev_texts)))
train_data = list(zip(train_texts,
                      [{'cats': cats} for cats in train_cats]))
print(train_texts[:5],train_cats[:5])

Loading IMDB data...
Using 2000 examples (1600 training, 400 evaluation)
('My mother worked with Dennis L. Raider for eleven years, not to mention shared an office with him. When it was announced he was BTK, she was shocked. The whole day was just her telling stories about how she never would have seen him as the Wichita Killer. I\'ve heard her re-tell them many times. I\'ve inquired her about a lot of things, and gone to all the interviews that she was asked to go to. I\'ve read the entire book written about Raider, Wichita is my hometown and I was surprised that such a thing could happen in Kansas.\n\n\n\nThere was another BTK movie on TV not too long ago, and I thought this one would have been better at portraying Dennis\' killings, maybe even have some intelligent touches to his motives.\n\n\n\nI\'m going to be very blunt with the flaws in this movie. This is based on my mom\'s portrayal of him, all my readings on him, and the video tapes I\'ve seen of him talking.\n\n\n\nFirst of 

In [35]:
nlp = spacy.load('en')

tokens = [nlp(x) for x in train_texts]
print(tokens[0])

My mother worked with Dennis L. Raider for eleven years, not to mention shared an office with him. When it was announced he was BTK, she was shocked. The whole day was just her telling stories about how she never would have seen him as the Wichita Killer. I've heard her re-tell them many times. I've inquired her about a lot of things, and gone to all the interviews that she was asked to go to. I've read the entire book written about Raider, Wichita is my hometown and I was surprised that such a thing could happen in Kansas.



There was another BTK movie on TV not too long ago, and I thought this one would have been better at portraying Dennis' killings, maybe even have some intelligent touches to his motives.



I'm going to be very blunt with the flaws in this movie. This is based on my mom's portrayal of him, all my readings on him, and the video tapes I've seen of him talking.



First of all, the camera angles were horrible. It looked as though it had been shot on a home video cam

In [36]:
print(tokens[0][0].vector)

[ -7.88929641e-01   2.49472904e+00   1.00183964e-01   5.38341808e+00
  -2.34359550e+00  -6.48356974e-03   2.08560324e+00   4.17754292e-01
  -2.53991461e+00   1.17190552e+00  -5.45085430e-01   1.99631667e+00
   2.20726848e+00  -2.87431192e+00  -9.07552123e-01  -7.13188410e-01
   2.00203657e+00   1.94827056e+00   1.38894677e+00  -2.59848163e-02
   2.65874267e+00   1.89830577e+00   1.02724814e+00   3.62629080e+00
   5.48710227e-01  -8.50266159e-01  -1.53428459e+00   1.15709448e+00
   1.81238890e+00   1.45786011e+00   1.91748834e+00   5.52694273e+00
   9.55302954e-01   4.26679850e-01   6.33236766e-02   1.09611678e+00
   3.92184758e+00   3.99406999e-02  -3.19887996e+00   9.80772257e-01
  -1.04980576e+00  -1.30675709e+00  -1.55589318e+00  -1.47749376e+00
  -6.66910589e-01  -2.24673295e+00  -2.21372724e-01   1.35512769e+00
  -7.36700118e-01  -8.31505537e-01   7.00726628e-01   4.04854965e+00
  -1.48289466e+00  -2.10138655e+00  -2.37878847e+00   2.66663766e+00
   4.54227656e-01  -1.80881274e+00

In [38]:
vectors=[]
for x in tokens:
    vectors.append([w.vector/w.vector_norm for w in x])